# ECM Client Setup for Google Colab

This notebook sets up the ECM factorization client on Google Colab.

**Repository:** https://github.com/kyleaskine/ecm-wrapper

## ⚠️ IMPORTANT: Enable GPU Runtime FIRST!

**Before running any cells**, ensure you have a GPU runtime:

1. Click **Runtime** menu → **Change runtime type**
2. Set **Hardware accelerator** to **T4 GPU**
3. Click **Save**

This will disconnect your current session - that's expected!

## Instructions

1. **FIRST**: Enable T4 GPU runtime (see above)
2. Run the **GPU Check Cell** to verify GPU is available
3. Run the **Setup Cell** and enter your username
4. Use the **Test Cell** to verify everything works
5. Use the **Run ECM Cell** to get work from the server and submit results

**Note:** You'll need to re-run setup each time you start a new Colab session.

In [ ]:
# ============================================================
# GPU CHECK - Run this FIRST!
# ============================================================

import subprocess
import sys

print("=" * 60)
print("  GPU Runtime Check")
print("=" * 60)
print()

# Check for NVIDIA GPU
try:
    result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'],
                          capture_output=True, text=True, timeout=10)
    if result.returncode == 0 and result.stdout.strip():
        gpu_info = result.stdout.strip()
        print(f"✅ GPU DETECTED: {gpu_info}")
        print()
        print("You're good to go! Proceed to the Setup Cell below.")
    else:
        raise Exception("No GPU found")
except Exception as e:
    print("❌ NO GPU RUNTIME DETECTED!")
    print()
    print("You need to enable GPU runtime before continuing:")
    print()
    print("1. Click 'Runtime' menu at the top")
    print("2. Select 'Change runtime type'")
    print("3. Set 'Hardware accelerator' to 'T4 GPU'")
    print("4. Click 'Save'")
    print()
    print("⚠️  This will disconnect your session - reconnect and run this cell again.")
    print()
    sys.exit(1)  # Stop execution here

print("=" * 60)

In [ ]:
# ============================================================
# SETUP CELL - Run this first!
# ============================================================

import os
import subprocess

print("=" * 60)
print("  ECM Factorization Client - Google Colab Setup")
print("=" * 60)
print()

# ============================================================
# Step 1: User Configuration
# ============================================================
USERNAME = input("📝 Enter your username: ").strip()

# ECM work parameters
B1_VALUE = input("🔢 Enter B1 value (default: 11000000): ").strip()
B1_VALUE = B1_VALUE if B1_VALUE else "11000000"

PRIORITY_VALUE = input("⭐ Enter priority filter (default: 5): ").strip()
PRIORITY_VALUE = PRIORITY_VALUE if PRIORITY_VALUE else "5"

# Production API endpoint
API_ENDPOINT = "https://ecm.kyleaskine.com/api/v1"

print()
print("=" * 60)

# ============================================================
# Step 2: Clone Repository
# ============================================================
print("📦 Cloning ecm-wrapper repository...")
if os.path.exists("/content/ecm-wrapper"):
    !rm -rf /content/ecm-wrapper
!git clone -q https://github.com/kyleaskine/ecm-wrapper.git /content/ecm-wrapper
os.chdir("/content/ecm-wrapper/client")
print("✓ Repository cloned")

# ============================================================
# Step 3: Create data directory
# ============================================================
print("📁 Creating data directory...")
!mkdir -p data
print("✓ Data directory created")

# ============================================================
# Step 4: Download ECM Binary
# ============================================================
print("⬇️  Downloading ECM binary...")
ECM_DOWNLOAD_URL = "https://ecm.kyleaskine.com/downloads/ecm75/ecm.gz"
ECM_PATH = "/content/ecm"

try:
    !wget -q --show-progress {ECM_DOWNLOAD_URL} -O {ECM_PATH}.gz
    !gunzip -f {ECM_PATH}.gz
    !chmod +x {ECM_PATH}

    # Verify installation
    result = subprocess.run([ECM_PATH, "--version"],
                          capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        version_info = result.stdout.strip() or result.stderr.strip()
        print(f"✓ ECM binary installed: {version_info.split()[0] if version_info else 'OK'}")
    else:
        print("✓ ECM binary downloaded (version check returned non-zero)")
except Exception as e:
    print(f"⚠️  Note: {e}")
    print("   Binary downloaded, continuing setup...")

# ============================================================
# Step 5: Install Python Dependencies
# ============================================================
print("📚 Installing Python dependencies...")
!pip install -q requests pyyaml
print("✓ Dependencies installed (requests, pyyaml)")

# ============================================================
# Step 6: Create client.local.yaml
# ============================================================
print("⚙️  Creating client.local.yaml...")

local_config = f"""# Google Colab Local Configuration
# This file overrides settings in client.yaml

api:
  endpoint: "{API_ENDPOINT}"
  timeout: 30

client:
  username: "{USERNAME}"
  cpu_name: "Google-Colab"

programs:
  gmp_ecm:
    path: "{ECM_PATH}"
    gpu_enabled: true
    gpu_device: 0

# Logging configuration
logging:
  level: "INFO"
  file: "data/logs/ecm_client.log"
  console: true
"""

with open("client.local.yaml", "w") as f:
    f.write(local_config)

print("✓ Configuration file created")

# ============================================================
# Step 7: Setup Complete - Display Summary
# ============================================================
print()
print("=" * 60)
print("✅ SETUP COMPLETE!")
print("=" * 60)
print(f"Username:      {USERNAME}")
print(f"API Endpoint:  {API_ENDPOINT}")
print(f"CPU Name:      Google-Colab")
print(f"ECM Binary:    {ECM_PATH}")
print(f"Working Dir:   /content/ecm-wrapper/client")
print(f"GPU:           Enabled (device 0)")
print(f"B1 Value:      {B1_VALUE}")
print(f"Priority:      {PRIORITY_VALUE}")
print("=" * 60)
print()
print("🚀 Ready! Run the cells below to start factoring.")
print()
print("💡 TIP: You can upload numbers.txt to the data/ directory")
print("   using the file browser on the left, then run batch scripts.")
print()

## Test Cell

Run this to verify ECM is working correctly (no API submission).

In [ ]:
# Test factorization (no API submission)
print("🧪 Running test factorization...")
print("-" * 60)
!python3 ecm-wrapper.py --composite "123456789012345" --curves 5 --b1 11000 --no-submit
print("-" * 60)
print("✓ Test complete! Check output above for results.")

## Auto-Work Mode (Progressive Strategy)

This will continuously get work from the server (least ECM done first) and submit results.

Runs stage 1 only with GPU acceleration.

In [ ]:
# Auto-work mode with progressive strategy (Stage 1 only)
# Uses B1 and priority values configured during setup
print(f"Running auto-work with B1={B1_VALUE}, Priority={PRIORITY_VALUE} (Stage 1 only)")
print("-" * 60)
!python3 ecm-wrapper.py --auto-work --work-type progressive --stage1-only --b1 {B1_VALUE} --priority {PRIORITY_VALUE} -v

## Alternative: Run Auto-Work with Limited Count

Process a specific number of work assignments then stop.

In [ ]:
# Process 5 work assignments then stop
!python3 ecm-wrapper.py --auto-work --work-type progressive --work-count 5 --stage1-only --b1 {B1_VALUE} --priority {PRIORITY_VALUE} -v

## View Logs

In [ ]:
# View recent log entries
!tail -n 50 data/logs/ecm_client.log

## View Configuration

In [ ]:
# Display current configuration
!cat client.local.yaml

## View Factors Found

In [ ]:
# Display any factors that have been found
!cat data/factors_found.txt 2>/dev/null || echo "No factors found yet."